In [353]:
import pandas as pd

In [354]:
df  = pd.read_excel("../Datasets/Sehirler.xlsx")

In [355]:
df.head()

,Id,Name,Affected
0,1,Adana,1
1,2,Adıyaman,1
2,3,Afyonkarahisar,0
3,4,Ağrı,0
4,5,Amasya,0


In [356]:
df.isna().sum()

Id          0
Name        0
Affected    0
dtype: int64

In [357]:
df.columns

Index(['Id', 'Name', 'Affected'], dtype='object')

In [358]:
distances = pd.read_excel("../Datasets/ilmesafe.xlsx")

In [359]:
distances.fillna(0)

,İLLER ARASI MESAFE CETVELİ (01.01.2023),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82
0,İL PLAKA NO,İL ADI,ADANA,ADIYAMAN,AFYONKARAHİSAR,AĞRI,AMASYA,ANKARA,ANTALYA,ARTVİN,...,BATMAN,ŞIRNAK,BARTIN,ARDAHAN,IĞDIR,YALOVA,KARABÜK,KİLİS,OSMANİYE,DÜZCE
1,01,ADANA,0,337,578,973,603,492,547,1001,...,628,712,784,1037,1077,905,714,251,89,735
2,02,ADIYAMAN,337,0,915,642,627,732,884,715,...,297,464,1016,751,746,1137,946,210,244,966
3,03,AFYONKARAHİSAR,578,915,0,1320,596,255,287,1224,...,1207,1290,518,1361,1464,346,449,830,667,371
4,04,AĞRI,973,642,1320,0,740,1055,1424,364,...,365,424,1171,305,144,1363,1106,812,880,1192
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,77,YALOVA,905,1137,346,1363,629,407,610,1238,...,1403,1549,373,1347,1507,0,351,1116,994,171
78,78,KARABÜK,714,946,449,1106,371,216,735,958,...,1174,1358,86,1067,1250,351,0,925,803,180
79,79,KİLİS,251,210,830,812,640,711,798,885,...,467,550,995,920,915,1116,925,0,159,945
80,80,OSMANİYE,89,244,667,880,603,581,636,909,...,535,619,873,944,984,994,803,159,0,823


In [360]:
distances.head()

,İLLER ARASI MESAFE CETVELİ (01.01.2023),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82
0,İL PLAKA NO,İL ADI,ADANA,ADIYAMAN,AFYONKARAHİSAR,AĞRI,AMASYA,ANKARA,ANTALYA,ARTVİN,...,BATMAN,ŞIRNAK,BARTIN,ARDAHAN,IĞDIR,YALOVA,KARABÜK,KİLİS,OSMANİYE,DÜZCE
1,01,ADANA,NaN,337,578,973,603,492,547,1001,...,628,712,784,1037,1077,905,714,251,89,735
2,02,ADIYAMAN,337,NaN,915,642,627,732,884,715,...,297,464,1016,751,746,1137,946,210,244,966
3,03,AFYONKARAHİSAR,578,915,NaN,1320,596,255,287,1224,...,1207,1290,518,1361,1464,346,449,830,667,371
4,04,AĞRI,973,642,1320,NaN,740,1055,1424,364,...,365,424,1171,305,144,1363,1106,812,880,1192


In [361]:
population = pd.read_excel("../Datasets/population.xlsx")

In [362]:
population.head()

,IL,TOPLAM NUFUS,ERKEK NUFUS,KADIN NUFUS
0,Adana,2258718,1127516,1131202
1,Adıyaman,632459,318755,313704
2,Afyonkarahisar,736912,367399,369513
3,Ağrı,535435,276800,258635
4,Aksaray,423011,211232,211779


In [363]:
# population.reset_index(inplace=True,drop=False)

In [364]:
population.head(15)

,IL,TOPLAM NUFUS,ERKEK NUFUS,KADIN NUFUS
0,Adana,2258718,1127516,1131202
1,Adıyaman,632459,318755,313704
2,Afyonkarahisar,736912,367399,369513
3,Ağrı,535435,276800,258635
4,Aksaray,423011,211232,211779
5,Amasya,335494,167624,167870
6,Ankara,5663322,2805877,2857445
7,Antalya,2548308,1281943,1266365
8,Ardahan,96161,49811,46350
9,Artvin,169501,85523,83978


In [365]:
merged_df = pd.merge(df, population, on='Name')
merged_df

KeyError: 'Name'

In [ ]:
merged_df.sort_values('Population', ascending=False,inplace= True)
dist_centers = merged_df.head(15)


In [ ]:
def filter_dataframe(df, x_column, y_column):
    filtered_values = df.loc[df[y_column] == 0, x_column]
    return filtered_values

# Assuming you have a DataFrame called 'df' with columns 'x' and 'y'
dist_centers = filter_dataframe(dist_centers, 'Name','Affected')
# make dist_centers a dataframe
dist_centers = pd.DataFrame(dist_centers)
dist_centers
dist_df = pd.merge(dist_centers, merged_df, on='Name')
dist_df.drop(['Id'], axis=1, inplace=True)
dist_df
# add a colum "distribution" and set it to 1 for all rows
dist_df['Distribution'] = 1
dist_df

,Name,Affected,Population,Distribution
0,İstanbul,0,15462452,1
1,Ankara,0,5663322,1
2,İzmir,0,4394694,1
3,Bursa,0,3101833,1
4,Antalya,0,2548308,1
5,Konya,0,2250020,1
6,Kocaeli,0,1997258,1
7,Mersin,0,1868757,1
8,Manisa,0,1450616,1
9,Kayseri,0,1421455,1


In [ ]:
# join merge_df and dist_df on "Name" column but keep all the rows from merge_df 

city_df = pd.concat([merged_df, dist_df])
city_df.fillna(0, inplace=True)
city_df.drop_duplicates(subset=['Name'], inplace=True)
city_df

,Id,Name,Affected,Population,Distribution
32,34.0,İstanbul,0,15462452,0.0
4,6.0,Ankara,0,5663322,0.0
33,35.0,İzmir,0,4394694,0.0
14,16.0,Bursa,0,3101833,0.0
5,7.0,Antalya,0,2548308,0.0
...,...,...,...,...,...
77,79.0,Kilis,1,142792,0.0
27,29.0,Gümüşhane,0,141702,0.0
73,75.0,Ardahan,0,96161,0.0
60,62.0,Tunceli,0,83443,0.0
